In [1]:
import bs4 as bs
import requests as req

In [2]:
pagHtml = req.get("https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory#void").text

In [3]:
soup = bs.BeautifulSoup(pagHtml, 'html5lib')

In [4]:
table = soup.find("div", id="covid-19-pandemic-data")

In [5]:
print(table)

<div class="covid19-container-wrapper" id="covid-19-pandemic-data"><div id="covid19-container-marker" style="display:none"></div><div aria-label="COVID-19 pandemic by country and territory table" id="covid19-container" role="region" style="float:none;max-width:600px;" tabindex="0">
<table class="wikitable plainrowheaders sortable" id="thetable" style="text-align:right; margin:0 0 0.5em 1em;width:97%;">
<caption><div class="covid-show-table" style="font-size:80%;font-weight:500;"><a href="#covid-19-pandemic-data">[show all]</a></div><div class="covid-collapse-table" style="font-size:80%;font-weight:500;float: right;"><a href="#void">[collapse]</a></div><style data-mw-deduplicate="TemplateStyles:r992953826">.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-

In [6]:
trs = table.findAll('tr')
trsValues = []
trs[2].get_text()

'\n\n\nUnited States[f]\n\n32,610,374\n\n583,148\n\nNo data\n\n[17]\n'

In [7]:
trsValues = []
for tr in trs:
    trText = tr.get_text()
    trText = trText.replace(',', '.')
    trText2 = trText.replace('\n\n\n', '')  
    trText3 = trText2.replace('\n\n', ',')
    trText4 = trText3.replace('\n', '')
    trText5 = trText4.replace(trText4[trText4.find("["):trText4.find("]")+1], '').strip()
    trTextList = trText5.split(',') 
    trsValues.append(trTextList)

location = []
cases = []
deaths = []
recov = []

for x in range(len(trsValues)-3):
   location.append(trsValues[x+1][0])
   cases.append(trsValues[x+1][1])
   deaths.append(trsValues[x+1][2])
   recov.append(trsValues[x+1][3])

In [8]:
import pandas as pd
import numpy as np

In [9]:
casesDF = pd.DataFrame(np.column_stack([location, cases, deaths,recov ]), 
                               columns=['Location', 'Cases', 'Deaths', 'Recovery'])

In [10]:
casesDF.head(10)

,Location,Cases,Deaths,Recovery
0,World,152.875.042,3.202.699,89.625.669
1,United States,32.610.374,583.148,No data
2,India,19.557.457,215.542,15.992.271
3,Brazil,14.725.975,406.565,13.242.665
4,France,5.652.247,104.819,No data
5,Turkey,4.875.388,40.844,4.480.381
6,Russia,4.831.744,111.198,4.450.289
7,United Kingdom,4.420.201,127.538,No data
8,Italy,4.044.762,121.177,3.492.679
9,Spain,3.524.077,78.216,No data


In [11]:
casesDF.describe()

,Location,Cases,Deaths,Recovery
count,240,240,240,240
unique,240,233,193,217
top,Taiwan,4,0,No data
freq,1,4,23,21


In [22]:
def filterDf(dataFrame, column, values):
    return dataFrame.loc[dataFrame[column].isin(values)]

In [25]:
locations_filter = ['Brazil','World', 'India', 'Russia', 'Turkey' ]

casesDfFiltered = filterDf(casesDF, 'Location', locations_filter)

In [68]:
casesDfFiltered

,Location,Cases,Deaths,Recovery
0,World,152.875.042,3.202.699,89.625.669
2,India,19.557.457,215.542,15.992.271
3,Brazil,14.725.975,406.565,13.242.665
5,Turkey,4.875.388,40.844,4.480.381
6,Russia,4.831.744,111.198,4.450.289


In [133]:
def getValuesInColumns(dataFrame, columns):
    values = []
    for column in columns:
        column_values = []
        for value in casesDfFiltered[column]:
            column_values.append(value.replace('.','_'))
        values.append(column_values)
    return values

In [204]:
columns = ['Deaths', 'Recovery', 'Cases']

deathsAndRecovery = getValuesInColumns(casesDfFiltered, columns)

deaths = deathsAndRecovery[0]
recovery = deathsAndRecovery[1]
casesInFilteredLocationDf = deathsAndRecovery[2]

In [156]:
deaths

['3_202_699', '215_542', '406_565', '40_844', '111_198']

In [157]:
recovery

['89_625_669', '15_992_271', '13_242_665', '4_480_381', '4_450_289']

In [214]:
lethalities = []
inactiveCases = []
for i in range(len(deaths)):
    death_i = int(deaths[i]) 
    nCases = int(recovery[i])+death_i
    lethality_i = death_i/nCases
    lethalities.append(lethality_i)
    nCases = '{0:,}'.format(nCases).replace(',','.')
    inactiveCases.append(nCases)

In [215]:
lethalities

[0.03450129598314171,
 0.013298648003897873,
 0.029786661958220353,
 0.009033834856703659,
 0.02437757687350638]

In [216]:
inactiveCases

['92.828.368', '16.207.813', '13.649.230', '4.521.225', '4.561.487']

In [217]:
lethalities_round = []
for lethality in lethalities:
    lethality *= 100 
    lethalities_round.append(round(lethality,3))

In [218]:
lethalities_round

[3.45, 1.33, 2.979, 0.903, 2.438]

In [226]:
activeCases = []

for i in range(len(inactiveCases)):
   activeCase = '{0:,}'.format(int(casesInFilteredLocationDf[i])-int(inactiveCases[i].replace('.',''))).replace(',','.')
   activeCases.append(activeCase)

In [230]:
casesDfFiltered.loc[:,'Lethality'] = lethalities_round
casesDfFiltered.loc[:,'Inactive Cases'] = inactiveCases
casesDfFiltered.loc[:,'Active Cases'] = activeCases

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [231]:
casesDfFiltered

,Location,Cases,Deaths,Recovery,Lethality,Inactive Cases,Active Cases
0,World,152.875.042,3.202.699,89.625.669,3.450,92.828.368,60.046.674
2,India,19.557.457,215.542,15.992.271,1.330,16.207.813,3.349.644
3,Brazil,14.725.975,406.565,13.242.665,2.979,13.649.230,1.076.745
5,Turkey,4.875.388,40.844,4.480.381,0.903,4.521.225,354.163
6,Russia,4.831.744,111.198,4.450.289,2.438,4.561.487,270.257
